Importamos los paquetes

In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

Definimos los parámetros del modelo y del entrenamiento

In [4]:
input_size = 784    # img size = (28,28) ---> 28*28=784 in total
hidden_size = 500   # number of nodes at hidden layer
num_classes = 10    # number of output classes discrete range [0,9]
num_epochs = 20     # number of times which the entire dataset is passed throughout the model
batch_size = 100    # the size of input data took for one iteration
lr = 1e-3          # size of step

Descargamos los datos y los cargamos

In [7]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100.0%
100.0%
100.0%


In [8]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                      batch_size = batch_size,
                                      shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                     batch_size = batch_size,
                                     shuffle = False)

Definimos la clase para el modelo y lo generamos

In [9]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [10]:
net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
    net.cuda()

Definimos la función de coste (Loss) y el optimizador

In [11]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

Entrenamos el modelo

In [12]:
for epoch in range(num_epochs):
    for i ,(images,labels) in enumerate(train_gen):
        images = Variable(images.view(-1,28*28))  # Eliminar .cuda()
        labels = Variable(labels)  # Eliminar .cuda()
        
        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

    print('Epoch [%d/%d], Loss: %.4f' 
          %(epoch+1, num_epochs, loss.data))

Epoch [1/20], Loss: 0.1711
Epoch [2/20], Loss: 0.1295
Epoch [3/20], Loss: 0.0513
Epoch [4/20], Loss: 0.0745
Epoch [5/20], Loss: 0.1068
Epoch [6/20], Loss: 0.0148
Epoch [7/20], Loss: 0.0110
Epoch [8/20], Loss: 0.0188
Epoch [9/20], Loss: 0.0494
Epoch [10/20], Loss: 0.0028
Epoch [11/20], Loss: 0.0012
Epoch [12/20], Loss: 0.0051
Epoch [13/20], Loss: 0.0327
Epoch [14/20], Loss: 0.0040
Epoch [15/20], Loss: 0.0034
Epoch [16/20], Loss: 0.0598
Epoch [17/20], Loss: 0.0126
Epoch [18/20], Loss: 0.0020
Epoch [19/20], Loss: 0.0001
Epoch [20/20], Loss: 0.0090


Evaluamos el modelo

In [13]:
correct = 0
total = 0
for images,labels in test_gen:
    images = Variable(images.view(-1,28*28))
    labels = labels
    
    output = net(images)
    _, predicted = torch.max(output,1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 97.970 %
